# 10_likert — 리커트 분석

In [ ]:
%run 00_config.ipynb
import pandas as pd, numpy as np

def reverse(series, max_scale): return max_scale + 1 - series
def tnb_masks(series, scale):
    rule = TNB_RULE[scale]
    return series.isin(rule["bottom"]), series.isin(rule["neutral"]), series.isin(rule["top"])

df = pd.read_excel(DATA_PATH)
w = choose_weight(df)

# 역코딩(q18,q19)
for col, mx in LIKERT_REVERSE.items():
    if col in df.columns:
        df[col] = reverse(df[col], mx)

likert_all = [c for c in (LIKERT_7 + LIKERT_5) if c in df.columns]

rows = []
for col in likert_all:
    scale = 7 if col in LIKERT_7 else 5
    s = df[col]
    b,n,t = tnb_masks(s, scale)
    valid = s.notna()
    wv = w[valid]; wsum = wv.sum()
    wb = (w[b & valid]).sum()/wsum*100 if wsum>0 else np.nan
    wn = (w[n & valid]).sum()/wsum*100 if wsum>0 else np.nan
    wt = (w[t & valid]).sum()/wsum*100 if wsum>0 else np.nan
    ub = (b & valid).mean()*100 if valid.any() else np.nan
    un = (n & valid).mean()*100 if valid.any() else np.nan
    ut = (t & valid).mean()*100 if valid.any() else np.nan
    rows.append({"variable": col, "scale": scale, "reverse_applied": col in LIKERT_REVERSE,
                 "valid_n": int(valid.sum()),
                 "weighted_bottom2_%": round(wb,2), "weighted_neutral_%": round(wn,2),
                 "weighted_top2_%": round(wt,2), "weighted_NSS_pp": round((wt-wb),2) if pd.notna(wt) and pd.notna(wb) else np.nan,
                 "unweighted_bottom2_%": round(ub,2), "unweighted_neutral_%": round(un,2),
                 "unweighted_top2_%": round(ut,2), "unweighted_NSS_pp": round((ut-ub),2) if pd.notna(ut) and pd.notna(ub) else np.nan})

likert_summary = pd.DataFrame(rows).sort_values("variable")

# q5 합성 및 α
q5_items = [c for c in ["q5_1","q5_2","q5_3","q5_4","q5_5"] if c in df.columns]
alpha_q5 = cronbach_alpha(df[q5_items]) if q5_items else np.nan
df["q5_scale_mean"] = df[q5_items].mean(axis=1) if q5_items else np.nan

likert_summary.to_csv(OUT_DIR/"likert_summary.csv", index=False, encoding="utf-8-sig")
df[[c for c in likert_all]+(["q5_scale_mean"] if "q5_scale_mean" in df.columns else [])]\
  .to_csv(OUT_DIR/"likert_data_extracted.csv", index=False, encoding="utf-8-sig")

print("alpha(q5):", round(alpha_q5,3))
likert_summary.head(10)
